Run #1 in order of benchmarking

This file processes the existing processed datafile, adds laggs and filters for 2021, and then removes some columns, then returns the file in the benchmark folder. This file is later used by the BS file.

In [8]:
#TODO : Unable to import these functions in parent directory

def lag_features(df, features, seq_length):
    """Transforms a raw 2D dataframe of option data into 2D dataframe ofsequence data.
    Last 2 indexes per sequence are bid and ask price. The len(features)*seq_length
    features before are sequences of features"""
    df = df.sort_values(["Expire_date", "Strike", "Ttl"], ascending = [True, True, False])
    
    # Adding lag for naive benchmarking
    df["Naive"] = df["Price"].shift(1)

    for step in range(seq_length)[::-1]:
        for feature in features:
            df[feature + "-" + str(step)] = df[feature].shift(step)
    
    df["Check_strike"] = df["Strike"] == df["Strike"].shift(seq_length-1)
    df["Check_expire"] = df["Expire_date"] == df["Expire_date"].shift(seq_length-1)
    df = df[(df["Check_strike"] == True) & (df["Check_expire"] == True)]
    df = df.drop(["Check_strike", "Check_expire"], axis=1)
    #df[["Bid_strike_last", "Ask_strike_last"]] = df[["Bid_strike", "Ask_strike"]]
    #df[["Bid_last", "Ask_last"]] = df[["Bid", "Ask"]]
    df["Price_last"] = df["Price"]
    df = df.sort_values(["Quote_date"], ascending = [True])
    return df

    
def read_file(file):
    """Read a single file and return a dataframe"""
    return pd.read_csv(file, skipinitialspace=True)



from math import log, sqrt, pi, exp
from scipy.stats import norm
import pandas as pd
import tensorflow as tf

In [9]:
first_year = 2019
last_year = 2021
file = f"../data/processed_data/{first_year}-{last_year}_underlying-strike_only-price.csv"
df_options = read_file(file)

In [10]:
features = ["Underlying_last", "Strike", "Ttl", "Volatility", "R"]
seq_length = 5
num_features = len(features)
num_outputs = 1

df_options = lag_features(df_options, features, seq_length)

In [11]:
df_options = df_options[["Quote_date", "Underlying_last", "Strike", "Ttl", "Volatility", "R", "Price", "Naive"]]
df_options['Quote_date'] = pd.to_datetime(df_options.Quote_date, format='%Y-%m-%d')
df_options = df_options[df_options.Quote_date.dt.year == 2021]

# To be used for testing on for faster computing
#df_options_short = df_options[df_options["Quote_date"] == "2021-01-06"]

In [12]:
print(df_options.head())

        Quote_date  Underlying_last  Strike  Ttl  Volatility     R     Price  \
3102998 2021-01-04          3701.38  2800.0  144    0.185353  0.09   919.600   
3103091 2021-01-04          3701.38  3950.0  144    0.185353  0.09    69.045   
3104380 2021-01-04          3701.38  2400.0  438    0.185353  0.10  1310.100   
3104484 2021-01-04          3701.38  1500.0  529    0.185353  0.10  2143.910   
3097922 2021-01-04          3701.38  3625.0    2    0.185353  0.09    84.595   

            Naive  
3102998   970.450  
3103091    80.350  
3104380  1362.445  
3104484  2197.200  
3097922   139.010  


In [13]:
mse_bs = tf.keras.metrics.mean_squared_error(
    df_options["Price"], df_options["Naive"]
)

In [14]:
print("MSE: ", mse_bs)
print("RMSE: ", sqrt(mse_bs))

MSE:  tf.Tensor(764.9342226363356, shape=(), dtype=float64)
RMSE:  27.657444253515827


In [15]:
# Write to file
df_options.to_csv('benchmark_with_naive.csv', encoding='utf-8', index=False)